<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Recurrent Neural Networks (RNNs)

_Author: Riley Dallas (ATX)_

---


### Learning Objectives

- Learn how to train/test split for sequence data
- Use `TimeseriesGenerator` to prep our time series data for modeling
- Create a basic RNN model using `keras`

In [ ]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU
from keras.optimizers import Adam
from keras.preprocessing.sequence import TimeseriesGenerator

import matplotlib.pyplot as plt
%matplotlib inline

## Load data: AAPL stock prices and SEC filings
---

There are two datasets that we'll merge together:
- Apple's historical stock prices (ticker: AAPL)
- Their SEC filings:
  - Annual earnings reports: 10-K
  - Quarterly earnings reports: 10-Q
  - Company news and updates: 8-K

Load both in the cells below.

In [ ]:
# load prices

In [ ]:
# load filings

## `Date` columns
---

For both datasets, we need to:
1. Convert the `Date` column to be a `datetime` object
2. Since these are time series datasets, we need the indices to be the `Date` columns

In [ ]:
# set prices Date to datetime

In [ ]:
# set prices index to Date column

In [ ]:
# set filings Date to datetime

In [ ]:
# set filings index to Date column

## Data cleaning: Prices
---

For Apple's stock prices, we'll need to do a few things:
1. Drop the `Adj Close` column since we already have a `Close` price
2. De-trend the dataset using `.pct_change()`
3. Drop any `NaN`s that were created from step 2

In [ ]:
# Drop Adj Close column

In [ ]:
# Plot Close column

In [ ]:
# Convert prices to be percent changes from prior row

In [ ]:
# Drop nulls

In [ ]:
# Plot Close column again

## Create `Target` column
---

Our column of interest is the `Close` prices. We could build a regression model, where we predict the actual percentage change, or we could set this up as a binary classification problem:
- 1: The `Close` price was greater than or equal to the previous day
- 0: The `Close` price was lower than the previous day

We'll go with the latter option. In the cell below, let's create our binary `Target` column.

In [ ]:
# Create Target column

## Merge Prices with SEC filings
---

SEC filings tend to have a huge impact on stock prices. Because of this, we'll want to merge the prices and filings `DataFrames` together.

**NOTE** We'll want to do a left join, so that we don't lose any rows that aren't associated with any SEC filings.

In [ ]:
# Merge DataFrames

## Fill `NaN`s
---

After the merge, we now have a bunch of `NaN`s for rows that didn't have any SEC filings on that date. Let's fill those in with 0.

In [ ]:
# Fill Nans with 0

## Train/Test Split
---

Since we're dealing with time series data, we don't want the model to be able to see the future. 

**When using `train_test_split` on time series date, set `shuffle=False` so that the order is maintained.**

In [ ]:
# Create X and y

In [ ]:
# Train/Test Split

## Scaling
---

You want to scale your data for any model that uses gradient descent, which includes neural networks.

In [ ]:
# Scale

## `TimeseriesGenerator`
---

We'll now convert our data into series' using `TimeseriesGenerator`. The `length` parameter dictates how many rows will constitute a sample. The generator also creates our batches.

```python
train_sequences = TimeseriesGenerator(X_train_sc, y_train, length=3, batch_size=64)
```

In the cells below, convert your training and validation data into sequences.

In [ ]:
# Create training sequences

In [ ]:
# Create test sequences

## Modeling time!
---

Now it's time to design our RNN model. We'll have:

- 2 `GRU` layers
- 2 hidden `Dense` layers
- 1 `Dense` output layer

We're doing a binary classification problem. As a result, our output layer needs to be 1 neuron, with the `sigmoid` activation function. This way, our model predicts a probability.

In [ ]:
# Design RNN